In [1]:
import torch
from torch import nn, tensor, randn
import pandas
# import nltk # See https://www.nltk.org/data.html
from nltk import tokenize
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [9]:
# Translate data from original CSV file into strings

to_replace = {'’': '\''} #, '—', '-'

data = pandas.read_csv("data/glados-portal2.original.csv", sep="|", index_col=0)["d1"]
file = open("data/glados-portal2.csv", "w", encoding='utf-16')
for string in data:
    file.write(string.translate(str.maketrans(to_replace)) + "\n")

In [219]:
char_tokenize = lambda text: tokenize.simple.CharTokenizer().tokenize(text)
word_lower_tokenize = lambda text: [t.lower() for t in tokenize.WordPunctTokenizer().tokenize(text)]
word_tokenize = lambda text: tokenize.WordPunctTokenizer().tokenize(text)

def byte_pair_encode(data, merge_count, chars_to_ignore=None):
    token_set = set(char_tokenize(data))
    return byte_pair_encode_continue(data, token_set.copy(), merge_count, chars_to_ignore)

import threading

def _bpe_count_pairs(token, token_set, pair_counts, data):
    for B in token_set:
        pair = token + B
        if pair in token_set: continue
        pair_counts.append((pair, data.count(pair)))


def byte_pair_encode_continue(data, token_set, merge_count, chars_to_ignore=None):
    # print('starts with ', token_set)
    token_set -= (chars_to_ignore or {' ', '.', ','})
    while merge_count != 0:
        new_token_set = token_set.copy()
        pair_counts = []
        for A in token_set:
            for B in token_set:
                pair = A + B
                if pair in token_set: continue
                pair_counts.append((pair, data.count(pair)))
                max_freq = 0
                most_frequent = []
        for v in pair_counts:
            if v[1] > max_freq:
                max_freq = v[1]
                most_frequent = [v[0]]
            elif v[1] == max_freq:
                most_frequent.append(v[0])
        # if len(most_frequent) > 0: print(most_frequent)
        for t in most_frequent:
            new_token_set.add(t)
            merge_count -= 1
            if merge_count == 0: break
        # print('+ ', new_token_set - token_set)
        token_set = new_token_set
    return token_set | (chars_to_ignore or {' ', '.', ','})

def bpe_tokenize(text):
    result = []
    words = text.split(' ')
    for w in words:
        while len(w) > 0:
            for i in range(len(w)):
                i = len(w) - i
                # print(w[:i])
                if w[:i] in token_to_id:
                    # print('found')
                    result.append(w[:i])
                    w = w[i:]
                    # print(w, 'still')
                    break
        result.append(' ')
    return result

# T (Time) -- размер контекстного окна
get_x = lambda data, T, pos: data[pos : pos + T]
get_y = lambda data, T, pos: data[pos + 1 : pos + T + 1]
# get_x_y = lambda data, T, pos: data[pos : pos + T], data[pos + 1 : pos + T + 1]

# B -- batch size
def get_batch(data, B, T):
    positions = torch.randint(len(data) - T, (B,))
    batch_x = torch.stack([get_x(data, T, pos) for pos in positions])
    batch_y = torch.stack([get_y(data, T, pos) for pos in positions])

In [215]:
test = byte_pair_encode(text, 20)
print(len(test))
print(test)
# print(bpe_tokenize("Did you tested it, don't you?"))

89
{'G', 'z', '2', '”', '?', 'te', 'yo', 'V', 'x', '.', 'l', 'p', ':', ',', 'e', '4', '–', 'R', 'W', 'ng', 'Y', 'an', '0', 'A', 'r', '5', 'f', 'T', 'b', 's', 'F', 'N', 'you', 'g', 'u', '[', ';', 'ing', 'm', 'in', 'M', '3', '—', 'he', 'K', 'd', '$', 'J', 'U', 'o', 'w', 'E', 'i', 'ou', 'er', 'st', 'y', '!', 't', 'on', 'v', 'D', 'th', 'I', 'H', ']', 'O', 'q', 'L', '1', 'S', 'the', 're', 'at', 'P', 'B', 'C', '7', 'k', 'j', '-', 'h', 'n', 'c', "'", 'a', ' ', '\n', 'es'}


In [ ]:
with open("data/glados-portal2.csv", "r", encoding='utf-16') as f:
    tokens = word_tokenize(f.read())

vocab = list(set(tokens))
C = len(vocab)

id_to_token = vocab
token_to_id = { t:i for i, t in enumerate(vocab)}

torch.manual_seed(0)

class DigramLM(nn.Module):
    def __init__(self, vocab):
        super.__init__()
        self.token_embedding_table = nn.Embedding(C, C)
    
    # def forward(self, input, targets):

# class LSTMLM(nn.Model):

# class TransformerLM(nn.Model):


# embedding = nn.Embedding(len(token_set), 2)
# display(torch.LongTensor([1]))
# print(len(token_set))
# embed = embedding(torch.LongTensor([1]))
# # X = torch.tensor(dataX, dtype=torch.float32).reshape(n_patterns, seq_length, 1)
# # inputs = tensor()

# # ᵺ = 2
# input_size = len(token_set)
# hidden_size = 10
# # data_strings = file.readlines()
# layer_number = 2

# lstm = nn.LSTM(input_size, hidden_size, layer_number)

# print(lstm(randn(1, len(token_set))))

AttributeError: module 'torch.nn' has no attribute 'Model'

In [74]:
with open("data/glados-portal2.csv", "r", encoding='utf-16') as f:
    text = f.read()

In [161]:
# Сгенерировать и сохранить набор токенов кодированием пар байтов
import pickle
bpe_token_file = open(r'bpe_tokens.pkl', 'wb')
bpe_token_set = byte_pair_encode(text, 60)
pickle.dump(bpe_token_set, bpe_token_file)
bpe_token_file.close()

In [224]:
# Загрузить, дополнить и сохранить токены КПБ
import pickle
bpe_token_file = open(r'bpe_tokens.pkl', 'rb')
bpe_token_set = pickle.load(bpe_token_file)
bpe_token_file.close()
bpe_token_set_old = bpe_token_set
bpe_token_set = byte_pair_encode_continue(text, bpe_token_set, 40)
bpe_token_file = open(r'bpe_tokens.pkl', 'wb')
pickle.dump(bpe_token_set, bpe_token_file)
bpe_token_file.close()
print('+', bpe_token_set - bpe_token_set_old)

+ {'ug', 'range', '.', 'un', ',', 'ect', 'ess', 'Bl', 'res', 'ag', 'com', 'ive', 'sh', 'ig', 'rang', 'op', 'can', 'pa', 'ba', 'ir', 'ht', 'ght', 'fe', 'hing', 'am', 'pr', 'here', ' ', 'abo'}


In [147]:
# Загрузить токены КПБ
import pickle
bpe_token_file = open(r'bpe_tokens.pkl', 'rb')
bpe_token_set = pickle.load(bpe_token_file)
bpe_token_file.close()
list(bpe_token_set)[:10]

['la', 'G', 'z', 'te', 'ne', 'est', 'x', 'us', 'be', 'p']

In [166]:
print(len(bpe_token_set))
print(list(bpe_token_set))

124
['G', 'z', '2', '”', '?', 'te', 'ne', 'yo', 'est', 'en', 'V', 'x', '.', 'us', 'be', 'l', 'p', 'co', ':', ',', 'as', 'me', 'e', '4', '–', 'R', 'W', 'el', 'ng', 'Y', 'nd', 'an', '0', 'A', 'r', 'hi', 'hat', '5', 'f', 'T', 'ea', 'b', 's', 'N', 'F', 'you', 'om', 'g', 'u', '[', ';', 'ing', 'm', 'ar', 'in', 'M', '3', '—', 'tes', 'nt', 'he', 'K', 'it', 'd', '$', 'se', 'or', 'J', 'll', 'U', 'o', 'w', 'E', 'ou', 've', 'er', 'i', 'st', 'y', 'ra', '!', 't', 'on', 'ur', 'v', 'th', 'D', 'I', 'H', ']', 'no', 'ri', 'le', 'to', 'O', 'q', 'L', '1', 'ed', 'is', 'S', 'al', 'test', 'the', 're', 'at', '7', 'B', 'P', 'C', 'ha', 'k', 'j', '-', 'h', 'n', 'ti', 'c', 'ma', "'", 'a', ' ', '\n', 'es']


In [246]:
# tokens = list(bpe_token_set | set(word_tokenize(text)))
tokens = list(bpe_token_set)
vocab =  sorted(tokens)
vocab = ['<PAD>', '<UNK>'] + vocab
vocab_size = len(vocab)

token_to_id = {t:i for i,t in enumerate(vocab)}
id_to_token = {i:t for t,i in token_to_id.items()}

torch.manual_seed(0)

tokenize_func = bpe_tokenize

def tokens_to_text(token_ids, id_to_token):
    return " ".join([id_to_token.get(token_id, "<UNK>") for token_id in token_ids])


class TextDataset(Dataset):
    def __init__(self, text, token_to_id, seq_length=20):
        self.token_ids = [token_to_id.get(t, token_to_id['<UNK>']) for t in tokenize_func(text)]
        self.seq_length = seq_length
        
    def __len__(self):
        return len(self.token_ids) - self.seq_length
    
    def __getitem__(self, idx):
        input_seq = self.token_ids[idx:idx+self.seq_length]
        target_seq = self.token_ids[idx+1:idx+self.seq_length+1]
        return torch.tensor(input_seq), torch.tensor(target_seq)

dataset = TextDataset(text, token_to_id)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [227]:

class SimpleLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim=64, hidden_size=128):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, x, hidden=None):
        x = self.embed(x)
        out, hidden = self.lstm(x, hidden)
        return self.fc(out), hidden
    
model = SimpleLSTM(vocab_size)
device = torch.device('cuda')
model.to(device)
criterion = nn.CrossEntropyLoss()

In [5]:
# загрузка
checkpoint = torch.load('text_lstm_model.pth')
model.load_state_dict(checkpoint['model_state'])
token_to_id = checkpoint['token_to_id']
id_to_token = checkpoint['id_to_token']

In [235]:
# обучение
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train(model, dataloader, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs, _ = model(inputs)
            loss = criterion(outputs.view(-1, outputs.size(-1)), targets.view(-1))
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader):.4f}')

train(model, dataloader, epochs=4)

Epoch 1/4, Loss: 0.7068
Epoch 2/4, Loss: 0.6889
Epoch 3/4, Loss: 0.6739
Epoch 4/4, Loss: 0.6605


In [244]:

# Генерация текста
def generate_text(model, start_text, token_to_id, id_to_token, max_length=100):
    model.eval()
    tokens = tokenize_func(start_text)
    token_ids = [token_to_id.get(t, token_to_id['<UNK>']) for t in tokens]
    hidden = None
    
    for _ in range(max_length):
        input_tensor = torch.tensor([token_ids[-1]]).unsqueeze(0).to(device)
        with torch.no_grad():
            output, hidden = model(input_tensor, hidden)
        
        probs = torch.softmax(output.squeeze(), dim=-1)
        next_token = torch.multinomial(probs, num_samples=1).item()
        token_ids.append(next_token)
    
    return ''.join([id_to_token[t] for t in token_ids])

# Пример использования
generated = generate_text(model, "You are", token_to_id, id_to_token)
print("\nСгенерированный текст:", generated)



Сгенерированный текст: You are people noticles.
That is first failure. The killing me. By the way. Enjoy these tests break from the testing courses, the last test? I'm looking at your partner the 


In [1]:
#Сохранение модели
torch.save({
    'model_state': model.state_dict(),
    'token_to_id': token_to_id,
    'id_to_token': id_to_token
}, 'text_lstm_model.pth')

NameError: name 'torch' is not defined